
# Proyecto: Implementación de Grafos con BFS y DFS

Implementación del TDA Grafo en Python usando Programación Orientada a Objetos.
- **Algoritmos:** BFS (Búsqueda en Anchura) y DFS (Búsqueda en Profundidad) implementados desde cero.
- **Visualización:** Se imprime la estructura del árbol (Padres e Hijos) en la terminal.


In [8]:
import time
from collections import deque

print("Librerías importadas (time, deque).")

Librerías importadas (time, deque).



### Definición de Clases (TDA Grafo) (Código)

In [9]:
class Nodo:
    def __init__(self, id, valor=None):
        self.id = id
        self.valor = valor
        self.vecinos = [] # Lista de objetos Arista

    def __repr__(self):
        # Ayuda a que se impriman bien los nodos
        return f"Nodo({self.id})"

class Arista:
    def __init__(self, origen, destino, peso=1):
        self.origen = origen
        self.destino = destino # Guardamos el ID del destino
        self.peso = peso

class Grafo:
    def __init__(self, dirigido=False):
        self.nodos = {} # Diccionario: id -> Objeto Nodo
        self.dirigido = dirigido
        print("Objeto Grafo creado.")

    def agregar_nodo(self, id, valor=None):
        if id not in self.nodos:
            self.nodos[id] = Nodo(id, valor)

    def agregar_arista(self, origen_id, destino_id, peso=1):
        # Aseguramos que los nodos existan
        self.agregar_nodo(origen_id)
        self.agregar_nodo(destino_id)

        # Crear arista origen -> destino
        arista_ida = Arista(origen_id, destino_id, peso)
        self.nodos[origen_id].vecinos.append(arista_ida)

        # Si el grafo NO es dirigido, creamos la arista inversa
        if not self.dirigido:
            arista_vuelta = Arista(destino_id, origen_id, peso)
            self.nodos[destino_id].vecinos.append(arista_vuelta)




### Métodos de Algoritmos (BFS, DFS) (Código)

In [10]:
# --- MÉTODO AUXILIAR PARA RECONSTRUIR RUTA ---
def _reconstruir_camino(self, arbol_padres, inicio, fin):
    camino = []
    actual = fin
    while actual is not None:
        camino.append(actual)
        if actual == inicio:
            break
        actual = arbol_padres.get(actual)

    if actual != inicio and fin != inicio:
        return []

    return camino[::-1] # Invertir lista para que vaya Inicio -> Fin

# Añadimos el método a la clase
Grafo._reconstruir_camino = _reconstruir_camino


# --- ALGORITMO BFS (Con reconstrucción de path) ---
def bfs(self, inicio_id, meta_id):
    if inicio_id not in self.nodos:
      return None

    start_time = time.time()

    cola = deque([inicio_id])
    visitados = {inicio_id: True}
    arbol_bfs = {inicio_id: None} # Diccionario de padres
    distancias = {inicio_id: 0}

    encontrado = False

    while cola:
        actual_id = cola.popleft()

        if actual_id == meta_id:
            encontrado = True
            # En BFS, podemos parar aquí para la ruta más corta
            break

        nodo_actual = self.nodos[actual_id]

        for arista in nodo_actual.vecinos:
            vecino_id = arista.destino

            if vecino_id not in visitados:
                visitados[vecino_id] = True
                arbol_bfs[vecino_id] = actual_id
                distancias[vecino_id] = distancias[actual_id] + 1
                cola.append(vecino_id)

    ruta = self._reconstruir_camino(arbol_bfs, inicio_id, meta_id if encontrado else None)
    tiempo_total = time.time() - start_time

    return {
        "algoritmo": "BFS",
        "ruta_mas_corta": ruta,
        "distancias": distancias,
        "arbol_padres": arbol_bfs, # Esto es el "Árbol BFS"
        "tiempo": tiempo_total,
        "encontrado": encontrado
    }

# Añadimos el método a la clase
Grafo.bfs = bfs


# --- ALGORITMO DFS (Con profundidad y árbol DFS) ---
def _dfs_recursivo(self, actual_id, meta_id, visitados, arbol_dfs, profundidad_actual, info_profundidad):
    visitados.add(actual_id)

    if profundidad_actual > info_profundidad["max_profundidad"]:
        info_profundidad["max_profundidad"] = profundidad_actual
        info_profundidad["nodo_mas_profundo"] = actual_id

    es_meta = (actual_id == meta_id)
    encontrado_en_hijos = False

    nodo_actual = self.nodos[actual_id]
    for arista in nodo_actual.vecinos:
        vecino_id = arista.destino

        if vecino_id not in visitados:
            arbol_dfs[vecino_id] = actual_id

            resultado_hijo = self._dfs_recursivo(vecino_id, meta_id, visitados, arbol_dfs, profundidad_actual + 1, info_profundidad)

            if resultado_hijo:
                encontrado_en_hijos = True

    return es_meta or encontrado_en_hijos

# Wrapper principal para DFS
def dfs(self, inicio_id, meta_id):
    if inicio_id not in self.nodos: return None

    start_time = time.time()

    arbol_dfs = {inicio_id: None}
    visitados = set()
    info_profundidad = {"max_profundidad": 0, "nodo_mas_profundo": inicio_id}

    encontrado = self._dfs_recursivo(inicio_id, meta_id, visitados, arbol_dfs, 0, info_profundidad)

    ruta_meta = self._reconstruir_camino(arbol_dfs, inicio_id, meta_id) if encontrado else []
    camino_profundo = self._reconstruir_camino(arbol_dfs, inicio_id, info_profundidad["nodo_mas_profundo"])
    tiempo_total = time.time() - start_time

    return {
        "algoritmo": "DFS",
        "ruta_a_meta": ruta_meta,
        "camino_mas_profundo": camino_profundo,
        "profundidad_maxima": info_profundidad["max_profundidad"],
        "arbol_dfs": arbol_dfs, # Esto es el "Árbol DFS"
        "tiempo": tiempo_total,
        "encontrado": encontrado
    }

# Añadimos los métodos a la clase
Grafo.dfs = dfs
Grafo._dfs_recursivo = _dfs_recursivo

print("Métodos BFS y DFS añadidos a la clase Grafo.")

Métodos BFS y DFS añadidos a la clase Grafo.



### Impresión en Terminal (Código)

In [11]:
def imprimir_arbol_terminal(arbol_padres, titulo="Árbol de Exploración"):
    """
    Imprime una representación del árbol (Padres -> Hijos) en la terminal.
    Toma el diccionario 'arbol_padres' (formato {hijo: padre}).
    """
    print(f"\n--- {titulo} (Vista de Terminal) ---")
    if not arbol_padres:
        print("El árbol está vacío.")
        return

    # Invertir el diccionario para tener un mapa de {padre: [lista_hijos]}
    hijos_por_padre = {}
    raiz = None

    for hijo, padre in arbol_padres.items():
        if padre is None:
            raiz = hijo
        else:
            if padre not in hijos_por_padre:
                hijos_por_padre[padre] = []
            hijos_por_padre[padre].append(hijo)

    if raiz is None:
        print("Error: No se encontró la raíz del árbol.")
        return

    print(f"Raíz: {raiz}")

    # Imprimir el árbol de forma ordenada para facilitar la lectura
    padres_ordenados = sorted(hijos_por_padre.keys())

    for padre in padres_ordenados:
        hijos = hijos_por_padre[padre]
        # Convertimos la lista de hijos a strings para el join
        hijos_str = [str(h) for h in hijos]
        print(f"  {padre} -> {', '.join(hijos_str)}")

    print("-" * (len(titulo) + 8))

print("Función 'imprimir_arbol_terminal' definida.")

Función 'imprimir_arbol_terminal' definida.



### Creación del Nuevo Grafo Experimental (Código)

In [12]:
# --- CREAR GRAFO SALAS DE HOSPITAL ---
g = Grafo(dirigido=False)
g.agregar_arista('R', 'E')
g.agregar_arista('R', 'A')
g.agregar_arista('A', 'E')
g.agregar_arista('E', 'C')
g.agregar_arista('E', 'F')
g.agregar_arista('F', 'C')
g.agregar_arista('C', 'X')
g.agregar_arista('C', 'L')
g.agregar_arista('L', 'U')
g.agregar_arista('U', 'I')
g.agregar_arista('I', 'H')
g.agregar_arista('H', 'U')
g.agregar_arista('U', 'Q')
g.agregar_arista('Q', 'P')
g.agregar_arista('U', 'M')
g.agregar_arista('P', 'H')

print("Creación del grafo salas de Hospital")
print(f"Nodos: {list(g.nodos.keys())}")

Objeto Grafo creado.
Creación del grafo salas de Hospital
Nodos: ['R', 'E', 'A', 'C', 'F', 'X', 'L', 'U', 'I', 'H', 'Q', 'P', 'M']


-----

### Ejecución BFS y Vista en Terminal (Código)

In [13]:
# --- EJECUCIÓN BFS ---
print("--- Ejecutando BFS (Ruta más corta) ---")
inicio = 'R'
fin = 'X'
res_bfs = g.bfs(inicio, fin)

# Imprimir resultados
print(f"¿Encontró salida?: {res_bfs['encontrado']}")
print(f"Ruta más corta: {res_bfs['ruta_mas_corta']}")
print(f"Distancia: {res_bfs['distancias'][fin]} pasos")
print(f"Tiempo BFS: {res_bfs['tiempo']:.6f} seg")

# Imprimir el Árbol BFS en la terminal
imprimir_arbol_terminal(res_bfs['arbol_padres'], "Árbol de Exploración BFS")

--- Ejecutando BFS (Ruta más corta) ---
¿Encontró salida?: True
Ruta más corta: ['R', 'E', 'C', 'X']
Distancia: 3 pasos
Tiempo BFS: 0.000028 seg

--- Árbol de Exploración BFS (Vista de Terminal) ---
Raíz: R
  C -> X, L
  E -> C, F
  R -> E, A
--------------------------------


-----

### Ejecución DFS y Vista en Terminal

In [14]:
# --- EJECUCIÓN DFS ---
print("\n--- Ejecutando DFS (Exploración profunda) ---")
inicio = 'R'
fin = 'X'
res_dfs = g.dfs(inicio, fin)

# Imprimir resultados
print(f"¿Encontró salida?: {res_dfs['encontrado']}")
print(f"Ruta DFS encontrada: {res_dfs['ruta_a_meta']}")
print(f"Camino de mayor profundidad: {res_dfs['camino_mas_profundo']}")
print(f"Profundidad máxima alcanzada: {res_dfs['profundidad_maxima']}")
print(f"Tiempo DFS: {res_dfs['tiempo']:.6f} seg")

# Imprimir el Árbol DFS en la terminal
imprimir_arbol_terminal(res_dfs['arbol_dfs'], "Árbol de Exploración DFS")


--- Ejecutando DFS (Exploración profunda) ---
¿Encontró salida?: True
Ruta DFS encontrada: ['R', 'E', 'C', 'X']
Camino de mayor profundidad: ['R', 'E', 'C', 'L', 'U', 'I', 'H', 'P', 'Q']
Profundidad máxima alcanzada: 8
Tiempo DFS: 0.000041 seg

--- Árbol de Exploración DFS (Vista de Terminal) ---
Raíz: R
  C -> F, X, L
  E -> A, C
  H -> P
  I -> H
  L -> U
  P -> Q
  R -> E
  U -> I, M
--------------------------------
